## 1. 한글 형태소 분석기 Nori 설치


한글 형태소 분석기 Nori 설치

    /home/pervinco/elasticsearch-8.8.0/bin/elasticsearch-plugin install analysis-nori
    sudo /home/pervinco/elasticsearch-8.8.0/bin/elasticsearch-plugin list

In [1]:
# 엘라스틱서치의 데몬 인스턴스 만들기
# Nori 설치 이전에 데몬을 생성하면 Nori가 바로 사용할 수 없음, 이때는 데몬 재실행 필요

import os
import json
import numpy as np
import pandas as pd

from subprocess import Popen, PIPE, STDOUT
from elasticsearch import Elasticsearch, helpers

es_server = Popen(['/home/pervinco/elasticsearch-8.8.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                #   preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# 데몬이 구동되었는지 확인 (세개의 daemon process가 있어야 함)
!ps -ef | grep elasticsearch

In [ ]:
username = 'elastic'
password = ''

# Elasticsearch 클라이언트 생성 (HTTP로 접속)
es = Elasticsearch(
    ['http://localhost:9200'],  # http로 변경
    basic_auth=(username, password)
)

# Elasticsearch 서버 정보 확인
resp = dict(es.info())
print(resp)


## 2. Nori analyzer의 활용

In [ ]:
# Nori를 사용하지 않을 경우 형태소 분석 결과 확인
# 기본적으로는 default analyzer인 "Standard analyzer"를 사용하게 됨 (공백으로 단어 분리, 소문자로 변환, 불용어 제거, 문장부호 기호 등 제거)
import pprint

result = es.indices.analyze(text ='모든 권력은 국민으로부터 나온다.')
pp = pprint.PrettyPrinter(indent=4, width=20)
pp.pprint(result)

result = es.indices.analyze(text ='홍대입구역너무복잡해')
pp = pprint.PrettyPrinter(indent=4, width=20)
pp.pprint(result)

In [ ]:
# Nori 사용할 경우 형태소 분석 결과 확인

result = es.indices.analyze(analyzer="nori", text ='모든 권력은 국민으로부터 나온다.')
pp = pprint.PrettyPrinter(indent=4, width=20)
pp.pprint(result)

result = es.indices.analyze(analyzer="nori", text ='홍대입구역너무복잡해')
pp = pprint.PrettyPrinter(indent=4, width=20)
pp.pprint(result)

- nori 토크나이져 활용 시 품사를 고려하여 token화 하는 것을 알 수 있다.

## 3. 위키데이터를 활용한 색인/검색 예시

In [ ]:
# 위키미디어로부터 kowiki 데이터를 다운로드 받음
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles1.xml-p1p82407.bz2
# 위키데이터의 노이즈를 제거하고 json 형태로 반환하는 코드를 참조
!git clone https://github.com/attardi/wikiextractor.git
# 다운로드 받은 샘플 위키 데이터를 전처리하여 검색의 입력으로 사용
# 결과는 elastic 폴더에 'extract_result/AA,AB,AC.../wiki_00..99'라는 새로운 폴더에 저장된다.(용량이 비슷하게 나눠서 저장됨)
# 변환결과 wiki_00 파일의 내용 샘플  {"id": "5", "revid": "641228", "url": "https://ko.wikipedia.org/wiki?curid=5", "title": "\uc9c0\...\ud130", "text": "\uc81c\...\ub2e4."}
!python -m wikiextractor.wikiextractor.WikiExtractor kowiki-latest-pages-articles1.xml-p1p82407.bz2 --json -o extract_result


In [ ]:
# 'wiki_dump_json_file'에 있는 JSON 파일 읽어들여 index_docs에 저장
index_docs = []
wiki_dump_json_file = './/extract_result/AA/wiki_00'

for line in open(wiki_dump_json_file, encoding="utf-8"):
    # JSON 데이터를 읽어들여 파이썬 딕셔너리로 변환
    json_data = json.loads(line)

    # 색인할 문서 목록에 추가
    index_docs.append(json_data)

index_docs[0]

In [8]:
# 색인을 위한 mapping 설정
setting = {
    "settings": {
        "analysis": {
            "analyzer": {
                "nori": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "decompound_mode": "mixed",
                    "filter": ["nori_posfilter"]
                }
            },
            "filter": {
                "nori_posfilter": {
                    "type": "nori_part_of_speech",
                    # 어미, 조사, 구분자, 줄임표, 지정사, 보조 용언 등
                    "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {"type": "text", "analyzer": "nori"},
            "text": {"type": "text", "analyzer": "nori"}
        }
    }
}

In [9]:
# Elasticsearch 색인/검색을 위한 공통 함수 정의

from elasticsearch import Elasticsearch, helpers
import json
import pprint as pp

def create_es_index(index, body):
    # 인덱스가 이미 존재하는지 확인
    if es.indices.exists(index=index):
        # 인덱스가 이미 존재하면 설정을 새로운 것으로 갱신하기 위해 삭제
        es.indices.delete(index=index)
    # 지정된 설정으로 새로운 인덱스 생성
    es.indices.create(index=index, body=body)

def delete_es_index(index):
    # 지정된 인덱스 삭제
    es.indices.delete(index=index)

def bulk_add(index, docs):
    # 대량 인덱싱 작업을 준비
    actions = [
        {
            '_index': index,
            '_source': doc
        }
        for doc in docs
    ]
    # Elasticsearch 헬퍼 함수를 사용하여 대량 인덱싱 수행
    return helpers.bulk(es, actions)

In [ ]:
# setting으로 설정된 내용으로 'test' 인덱스 생성
create_es_index("test", setting)

# 'test' 인덱스에 대량 색인화 수행
ret = bulk_add("test", index_docs)

# 결과 출력
print(ret)

In [ ]:
# title과 text 두 필드에서 검색
body = {
    "query": {
        "multi_match": {
            "query": "대한민국 대통령",
            "fields": ['title', 'text']  # wiki에서 json만들어질때 컬럼명이 제목은 title , 내용은 text
        }
    },
    "size": 10
}
res = es.search(index="test", body=body)

In [ ]:
# 결과 출력
for rst in res['hits']['hits']:
    print('score:', rst['_score'], 'source::', rst['_source'])